In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import os 
os.chdir("/content/drive/MyDrive/Thesis/Data/retrieved tweets")

In [3]:
import pandas as pd

folder = "07-01-2020"
ftype = "News"
ttype = "Real"
reply = False 

if ftype == "Claim" :
    n_id = 'claim_id'
elif ftype == "News" :
    n_id = 'news_id'

if reply :
    tweets = "tweets_replies"
    t_id = "reply_id"
else :
    tweets = "tweets"
    t_id = "tweet_id"
    
path_source = "/content/drive/MyDrive/Thesis/Data/retrieved tweets/" + folder + "/" + ftype + ttype + "COVID-19_" + tweets + "_" + folder +  ".txt"
path_target = "/content/drive/MyDrive/Thesis/Data/Filled Datasets/" + folder + "/" + ftype + ttype + "COVID-19_" + tweets + "_" + folder + "_hydr.csv"

df = pd.read_csv(path_source)

if reply :
    df.set_axis([n_id,'tweet_id','reply_id'], axis=1, inplace=True)
    df.tweet_id = df.tweet_id.astype(int)
else :
    df.set_axis([n_id,'tweet_id'], axis=1, inplace=True)
    df.tweet_id = df.tweet_id.astype(int)

In [4]:
print(df.count())
df.head()

news_id     54224
tweet_id    54224
dtype: int64


,news_id,tweet_id
0,1558,1278459263720009736
1,1558,1278390989032435713
2,1558,1278368893246091264
3,1558,1277714217186013188
4,1558,1277189612218339328


In [5]:
import requests
import os
import json

In [6]:
with open('/content/drive/MyDrive/Thesis/Data/retrieved tweets/07-01-2020/bearer_token') as f:
    bearer_token = f.read().strip()

In [7]:
def create_url(ids):
    tweet_fields = "tweet.fields=author_id,created_at,text,in_reply_to_user_id,public_metrics,possibly_sensitive"
    # Tweet fields are adjustable.
    # Options include:
    # attachments, author_id, context_annotations,
    # conversation_id, created_at, entities, geo, id,
    # in_reply_to_user_id, lang, non_public_metrics, organic_metrics,
    # possibly_sensitive, promoted_metrics, public_metrics, referenced_tweets,
    # source, text, and withheld
    ids = "ids=" + ids
    url = "https://api.twitter.com/2/tweets?{}&{}".format(ids, tweet_fields)
    return url

In [8]:
def bearer_oauth(r):
    # Method required by bearer token authentication.
    r.headers["Authorization"] = f"Bearer {bearer_token}"
    r.headers["User-Agent"] = "v2TweetLookupPython"
    return r

In [9]:
def connect_to_endpoint(url):
    response = requests.request("GET", url, auth=bearer_oauth)
    print(response.status_code)
    if response.status_code != 200:
        raise Exception(
            "Request returned an error: {} {}".format(
                response.status_code, response.text
            )
        )
    return response.json()

In [10]:
import time 

In [ ]:
ids = df[t_id].astype(str).drop_duplicates().to_list()
l = len(ids)
df2 = pd.DataFrame()
for i in range(0,l,100):
    idstr = ','.join(ids[i:i+100])
    url = create_url(idstr)
    json_response = connect_to_endpoint(url)
    df2 = pd.concat([df2,pd.DataFrame(json_response["data"])],axis=0)
    time.sleep(2)

In [ ]:
df2.id = df2.id.astype(int)
df2 = df2.rename({'id': t_id},axis=1)
print(df2.count())

In [ ]:
df2.head(10)

In [13]:
df3 = df.merge(df2,on=t_id,how='left')

In [ ]:
len(df3)

In [ ]:
df3.head(10)

In [ ]:
df3.count()

In [ ]:
df3.to_csv(path_target)